In [8]:
import pandas as pd
import numpy as np

train = pd.read_csv('./dataset/train.csv')
train.head()

,id,dt,type,target
0,00037f39cf,11,2,44.050
1,00037f39cf,12,2,50.672
2,00037f39cf,13,2,39.042
3,00037f39cf,14,2,35.900
4,00037f39cf,15,2,53.888


|特征字段|	字段描述|
|--|--|
|id	|房屋id|
|dt	|日标识|
|type	|房屋类型|
|target	|实际电力消耗，预测目标|

In [9]:
test = pd.read_csv("./dataset/test.csv")
test.head()

,id,dt,type
0,00037f39cf,1,2
1,00037f39cf,2,2
2,00037f39cf,3,2
3,00037f39cf,4,2
4,00037f39cf,5,2


In [11]:
# 3. 计算训练数据最近11-20单位时间内对应id的目标均值
target_mean = train[train["dt"] <= 20].groupby(["id"])["target"].mean().reset_index()

# 4. 将target_mean作为测试集结果进行合并
test = test.merge(target_mean, on=["id"], how="left")

# 5. 保存结果文件到本地
test[["id", "dt", "target"]].to_csv("submit.csv", index=None)